In [1]:
abstract class Absurd{
    def apply[A]: A
}

type Not[A] = A => Absurd

abstract class Or[A, B]{
    def apply[C](f: A => C, g: B => C): C
}

object Or{
    def applyL[A, B](a: A): Or[A, B] = new Or[A, B]{
        def apply[C](f: A => C, g: B => C): C = 
            f(a)
    }

    def applyR[A, B](b: B): Or[A, B] = new Or[A, B]{
        def apply[C](f: A => C, g: B => C): C = 
            g(b)
    }
    
    def elim[A, B, C](or: Or[A, B], f: A => C, g: B => C): C = 
        or[C](f, g)
}

abstract class And[A, B]{
    def apply[C](f: A => B => C): C
}

object And{
    def apply[A, B](a: A, b: B): And[A, B] = new And[A, B]{
        def apply[C](f: A => B => C): C = 
            f(a)(b)
    }
    
    def elimL[A, B](and: And[A, B]): A = 
        and(a => _ => a)
    
    def elimR[A, B](and: And[A, B]): B = 
        and(_ => b => b)
}



defined class Absurd
defined type Not
defined class Or
defined object Or
defined class And
defined object And

In [2]:
def proof1[A, B](or: Or[A, B]): Not[A] => B = 
    (n: Not[A]) => or[B]((a: A) => n(a)[B], (b: B) => b)

def proof2[A, B](n: Not[A]): A => B = 
    (a: A) => n(a)[B]

def proof3[A, B](i: A => Not[B]): (A => B) => Not[A] = 
    (i2: A => B) => (a: A) => i(a)(i2(a))

def proof4[A, B](n: Not[A => B]): Not[B] = 
    (b: B) => n((_: A) => b)

def proof5[A, B](n: Not[A]): Not[And[A, B]] = 
//    (and: And[A, B]) => n(And.elimL(and))
    n compose And.elimL

def proof6[A, B]: Not[And[A, Not[A]]] = 
    (a: And[A, Not[A]]) => And.elimR(a)(And.elimL(a))

def proof7[A, B](n: Not[And[A, B]]): A => Not[B] = 
    (a: A) => (b: B) => n(And(a, b))

def proof8[A, B](or: Or[A, B]): Or[B, A] = 
    or[Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

def proof9[A, B](n: Not[A Or B]): Not[A] And Not[B] = 
    And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

def proof10[A, B](and: Not[A] And Not[B]): Not[A Or B] = 
    or => or[Absurd](And.elimL(and), And.elimR(and))

def proof11[A](a: A And Not[A]): Absurd = 
    And.elimR(a)(And.elimL(a))


defined function proof1
defined function proof2
defined function proof3
defined function proof4
defined function proof5
defined function proof6
defined function proof7
defined function proof8
defined function proof9
defined function proof10
defined function proof11

In [3]:
trait ClassicalLogic{
    trait DN{
        def apply[A](nn: Not[Not[A]]): A
    }
    
    val dn: DN
    
    def proof1[A](f: Not[A] => A): A =
        dn((n: Not[A]) => proof11(And(f(n),n)))
}


defined trait ClassicalLogic